In [1]:
include("Axis_Aligned_Box.jl");
type Mondrian_Node
    parent::Nullable{Mondrian_Node}
    left::Nullable{Mondrian_Node}
    right::Nullable{Mondrian_Node}
    τ::Float64
    node_type::Array{Bool,1}        # node,leaf,root
    δ::Nullable{Int}                # split dimension
    ζ::Nullable{Float64}            # split position
    Θ::Nullable{Axis_Aligned_Box}   # data boxes B
    tab::Array{Int}
    c::Array{Int}
    Gₚ::Array{Float64}
end

function Mondrian_Node()
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      0,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64)
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,[true,false,false],
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

function Mondrian_Node(τ::Float64, node_type::Array{Bool,1})
    N = Mondrian_Node(Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      Nullable{Mondrian_Node}(),
                      τ,
                      node_type,
                      Nullable{Int}(),
                      Nullable{Float64}(),
                      Nullable{Axis_Aligned_Box}(),
                      Array{Int}(),
                      Array{Int}(),
                      Array{Float64}())
    return N
end

type Mondrian_Tree
    root::Nullable{Mondrian_Node}
    leaves::Array{Mondrian_Node,1}
end

function Mondrian_Tree()
    T = Mondrian_Tree(Nullable{Mondrian_Node}(),Array{Mondrian_Node,1}())
end

function Mondrian_Tree(N::Mondrian_Node)
    T = Mondrian_Tree(N,Array{Mondrian_Node,1}())
end

Mondrian_Tree

In [481]:
using Distributions

function Sample_Mondrian_Tree(λ,D)
    δ = []
    ζ = []
    n = 1:size(D,1)
    e = Mondrian_Node(0.0,[false,false,true])
    T = Mondrian_Tree(e)
    Θ = Axis_Aligned_Box(get_intervals(D[1]))
    e.Θ = Θ
    e.tab = zeros(size(unique(D[2]),1))
    e.c = zeros(size(unique(D[2]),1))
    e.Gₚ = zeros(size(unique(D[2]),1))
    Sample_Mondrian_Block(e, Θ, λ, δ, ζ, T)
    return T
end

function Sample_Mondrian_Block(j, Θ, λ, δ, ζ, tree)
    E = rand(Exponential(1/Linear_dimension(Θ)))
    println(E)
    if j.node_type[3]==true
        τₚ = 0
    else
        τₚ = (get(j.parent)).τ
    end
    if τₚ + E < λ
        d,x = sample_split_dimension(Θ)
        j.δ = d
        j.ζ = x
        j.τ = τₚ+E
        Θᴸ = copy(Θ)
        Θᴿ = copy(Θ)
        Θᴸ.Intervals[d,2]=x
        Θᴿ.Intervals[d,1]=x
        left = Mondrian_Node(0.0, [true,false,false])
        left.parent = j
        left.Θ = Θᴸ
        left.tab = zeros(size(j.tab))
        left.c = zeros(size(j.tab))
        left.Gₚ = zeros(size(unique(D[2]),1))
        right = Mondrian_Node(0.0, [true,false,false])
        right.parent = j
        right.Θ = Θᴿ
        right.tab = zeros(size(j.tab))
        right.c = zeros(size(j.tab))
        right.Gₚ=zeros(size(unique(D[2]),1))
        j.left = left
        j.right = right
        Sample_Mondrian_Block(left, Θᴸ, λ, δ, ζ, tree)
        Sample_Mondrian_Block(right, Θᴿ,λ, δ, ζ, tree)
    else
        j.τ = λ
        j.node_type = [false,true,false]
        push!(tree.leaves,j)
    end
end

function get_data_indices(Θ::Axis_Aligned_Box, D::Array{Float64,2})
    indices = []
    include = false
    for i in 1:size(D,1)
        for j in 1:size(Θ.Intervals,1)
            if (D[i,j] < Θ.Intervals[j,1] || D[i,j] > Θ.Intervals[j,2])
                include = false
                break
            end
            include = true
        end
        if (include)    
            push!(indices, i)
        end
    end
    return indices
end

function set_label_distribution!(j,Θ, D)
    indices = get_data_indices(Θ, D)
    for k in 1:length(unique(D[2]))
        j.Gₚ = length(D[2][D[2].==k])
    end
    j.Gₚ = j.Gₚ/(sum(j.Gₚ))
end

function initialize_posterior_leaf_counts!(Tree, D)
    X = D[1]
    Y = D[2]
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),X)
        y = Y[indices]
        for k in 1:length(leaf.c)
            leaf.c[k] = length(y[y.==k])
            leaf.tab[k] = min(leaf.c[k],1)
        end
    end
end

function initialize_posterior_counts!(Tree,D)
    initialize_posterior_leaf_counts!(Tree,D)
    for leaf in Tree.leaves
        j = leaf
        while true
            if j.node_type[2]==false
                for k in 1:length(j.c)
                    j.c[k] = get(j.left).tab[k]+get(j.right).tab[k]
                end
            end
            for k in 1:length(j.c)
                j.tab[k] = min(j.c[k],1)
            end
            if j.node_type[3]==true
                break
            else
                j = get(j.parent)
            end
        end
    end
end

function update_posterior_counts!(leaf,y)
    leaf.c[y] += 1
    l = leaf
    while true
        if l.tab[y] == 1
            return
        else
            if l.node_type[2] == false
                l.c[y] = get(l.left).tab[y]+get(l.right).tab[y]
            end
            l.tab[y] = min(l.c[y],1)
            if l.node_type[3] == true
                return
            else
                l = get(l.parent)
            end
        end
    end         
end

function update_posterior_counts_batch!(Tree,D)
    for leaf in Tree.leaves
        indices = get_data_indices(get(leaf.Θ),D[1])
        for y in D[2][indices,:]
            update_posterior_counts(leaf,y)
        end
    end
end

function compute_predictive_posterior_distribution!(Tree, γ=1)
    J = [get(Tree.root)]
    while (size(J,1) != 0)
        j = shift!(J)
        if (j.node_type[3]==true)
            p = ones(length(j.c))./length(j.c)
            d = exp(-γ*(j.τ))
        else 
            d = exp(-γ*(j.τ-get(j.parent).τ))
            p = get(j.parent).Gₚ
        end
        for k in 1:length(j.c)
            j.Gₚ[k] = (1/(sum(j.c)+1))*(j.c[k]-d*j.tab[k]+sum(j.tab)*p[k])
        end
        if j.node_type[2] == false 
            push!(J, get(j.left))
            push!(J, get(j.right))
        end
    end
end

function expected_discount(nⱼ, Δⱼ,γ=1)
    Δ = rand(Truncated(Exponential(1/nⱼ),0,Δⱼ),10000)
    return mean(exp.(-γ*Δ))
end

function predict!(T,x, γ=1)
    j = get(T.root)
    not_sep = 1
    s = zeros(size(j.c,1))
    while true
        if (j.node_type[3] == true)
            Δⱼ = j.τ
        else
            Δⱼ = j.τ - get(j.parent).τ
        end
        nⱼ=0
        for d in size(x,1)
            nⱼ += max(x[d]-get(j.Θ).Intervals[d,2],0) + max(get(j.Θ).Intervals[d,1]-x[d],0)
        end
        pⱼ = 1-exp(Δⱼ*nⱼ)
        if pⱼ > 0
            println("Yes")
            # x branches
            jₓ = Mondrian_Node()
            if (j == get(j.parent).left)
                get(j.parent).left = jₓ
            else
                get(j.parent).right = jₓ
            end
            jₓ.parent = get(j.parent)
            j.parent = jₓ
            jₓ.left = j
            jₓ.right = Mondrian_Node()
            d = expected_discount(nⱼ, Δⱼ, γ)
            for k in 1:length(j.c)
                jₓ.c[k] = min(j.c[k],1)
            end
            jₓ.tab = jₓ.c
            for k in 1:length(jₓ.c)
                jₓ.Gₚ[k] = 1/(sum(jₓ.c)+1)*(jₓ.c[k] - d*jₓ.tab[k]+d*sum(jₓ.tab)*get(jₓ.parent).Gₚ[k])
            end
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*jₓ.Gₚ[k]
            end
        end
        if j.node_type[2] == true
            for k in 1:length(s)
                s[k] += not_sep*(1-pⱼ)*j.Gₚ[k]
            end
            return s
        else
            not_sep = not_sep*(1-pⱼ)
            if x[get(j.δ)] <= get(j.ζ)
                j = get(j.left)
            else
                j = get(j.right)
            end
        end
    end
end

predict! (generic function with 2 methods)

In [492]:
function Fakedata(n,dim) 
    x = randn(n,dim)
    y = (sum(exp.(x)/(1+exp.(x)),2)).>0.5
    return 1.0*x,1*y[:,1]
end

d=4
X, Y = Fakedata(1000,d);
D = (X,Y.+1)

([-0.15442 0.931744 0.0742729 -0.167481; -0.461455 1.27141 -0.522623 0.0988499; … ; 1.81798 -1.24895 0.895447 0.164541; -1.49271 1.61533 0.293295 0.320159], [1, 1, 2, 1, 1, 1, 2, 1, 1, 1  …  1, 1, 2, 1, 2, 2, 1, 2, 2, 2])

In [503]:
MT = Sample_Mondrian_Tree(0.15,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

0.015717918642957577
0.013983244180441022
0.02195897699980682
0.005758629833878816
0.029293780937704853
0.0317188767317077
0.2900261515246122
0.08879750933948319
0.2461534303084561
0.1860442247054495
0.03299717039030958
0.00946121308765711
0.4482246963316456
0.004970041041804149
0.12424260130821337
0.05050348114022164
0.03868731655339767
0.15150296583985445
0.1044326685487309
0.018937556826132976
0.07245706181152427
0.18330714622184116
0.03148378513254584
0.3487882286003075
0.009036706263279383
0.11700165998928057
0.05000435496257411
0.08804816691394077
0.022324783313956202
0.008117957662418717
0.03241871621832449
0.2683063421235565
0.013868597102824152
0.07857288480437916
0.06099296130109674
0.008215686940918163
0.08565007867456897
0.012430682536259863
0.15755192397640932
0.057242996209670216
0.02772557973966809
0.005231631302959026
0.005285342011968803
0.02287350458865176
0.029627429459480083
0.12238510011888716
0.09913761132275599
0.016024479498107362
0.03045039196984418


In [499]:
using MLBase
pred = []
for i in 1:size(D[1],1) 
    p = predict!(MT,D[1][i,:],10*size(D[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println(correctrate(D[2],convert(Array{Int,1},pred)))
println(unique(pred))

0.681
[1, 2]


In [504]:
length(MT.leaves)

25

In [505]:
for leaf in MT.leaves
    println(leaf.c)
end

[1, 0]
[3, 0]
[0, 0]
[3, 0]
[0, 0]
[3, 0]
[4, 3]
[0, 0]
[4, 4]
[0, 0]
[0, 0]
[0, 0]
[0, 1]
[0, 0]
[522, 229]
[33, 119]
[0, 17]
[0, 3]
[0, 16]
[0, 10]
[0, 23]
[0, 1]
[0, 0]
[0, 1]
[0, 0]


In [487]:
include("load_titanic.jl")
x_train, y_train, x_test, y_test = load();
D = (x_train, y_train.+1)
D_test = (x_test,y_test.+1)
MT = Sample_Mondrian_Tree(0.03,D);
initialize_posterior_counts!(MT,D);
compute_predictive_posterior_distribution!(MT,10*size(D[1],2));

0.0004805162204639552
0.00045161277264253713
0.0003460641474184446
4.487469061512529e-5
0.0003741459839346572
0.0026048720180316645
0.001145137881601144
0.003522295263799767
0.009322564788605905
0.003649438134214123
0.01982221677144962
0.005901709795463851
0.013919811779086196
0.00024302186661376876
0.05027908153487941
0.014467057050334346
0.010019277654263095
0.004826255890635328
0.0180976673177186
0.001103219499419837
0.011898047112416304
0.0162022003866344
0.009498894218690265
0.0006785145394882228
0.006852793648960408
0.004029960823158865
0.01780746072978928
0.014107439523216738
0.013707559371176174
0.006152568959360614
0.003602847076412744
0.019381556981275777
0.0016577716663590462
0.0012059755305653924
0.0011709505289279084
0.0069309809877529
0.00675149081911717
0.00529063072296893
0.0035416763123094605
0.021922274727518218
0.0007713043292618111
0.003442234669432775
0.0023252896672863433
0.003060987332523735
0.017334186901701774
0.02589576590917402
0.0033672846160595266
0.0119327

0.011163786091705676
0.004058590981887407
0.036709321561909876
0.03482523708903597
0.0032616633499351158
0.005021757698557397
0.021240186185142354
0.0035820813689411797
0.0008748431157717606
0.02950870633169957
0.037152698442957044
0.003181909863246433
0.053752995017301165
0.021025248203276323
0.009867198325147224
0.0534740976585289
0.1574955147143177
0.0017406105835549841
0.0028149921441594497
0.005020568269980984
0.004106540754859411
0.0635601459548692
0.024449562278366692
0.11615999636443979
0.007224399489273082
0.09132640530204948
0.003265093929470756
0.05472528273317822
0.030687574411110023
0.001172560939966412
0.0014009964459849975
0.0037882444651524843
0.011532247939731536
0.0010847780233132515
0.00988929758480891
0.012198044910917371
3.499003000705152e-6
0.00021175748279867425
0.002876442912521951
0.012254159985435706
0.017907896024667788
0.0019777856804451617
0.000678433849289063
0.006826054968651835
0.0029444068796810824
0.009756562445561114
0.004813637962421697
0.00147682026

0.00034247343475274914
0.02883313661191609
0.009776668737695825
0.0018604257215752993
0.012108567139449904
0.0018862018421123702
0.008817431391000932
0.00947682906078188
0.0009033577371203592
0.005375147952859372
0.0007011654717745141
0.001666829441725756
0.0011954843222199663
0.00555444979562118
0.0012063196769121355
0.014041796820843704
0.0038215945904395802
0.004230826812387009
0.001096786326320744
0.0005720844949196748
0.0015409921437322207
0.00553482138810463
0.014496597094332292
0.013239827810192915
0.008329814504271971
0.0038021524328924316
0.004491098283972342
0.004152797347941766
0.001597236650645196
0.07672334067265303
0.005125450333773451
0.004996789540746407
0.008558241962349635
0.0021746679435725325
0.003994031789992522
0.0046556816260235325
0.00021747729072911704
0.002952919629796293
0.0029888492010952607
0.006787296053403348
0.005923799360787782
0.02300450048701207
0.0006615628126797035
0.0051243777651868595
0.0027478541619846385
0.0009902590521930956
0.03421476741899909

0.009730055987393635
0.002410866516422295
0.004372179035706232
0.010567995563126254
0.015682607865922227
0.0030626049431899936
0.004162552072498282
0.016127945931039223
0.028045951945297944
0.0016542304043504496
0.004898816369113795
0.010625554836421535
0.0024485292087966753
0.005355754011766097
0.01199891237053236
0.0016684888869027234
0.021068119003731643
0.01627271759416034
0.09952032798919273
0.0035656633026889383
0.006582884069866299
0.0039076608518870985
0.009126304288814325
0.007614355950859219
0.007512568547047524
0.003904590189255906
0.0019829710686910815
0.0024166097041144513
0.004458691654098449
0.0012229676382076876
0.01183182689352618
0.001258232947265485
0.0019266587858315152
0.020204946148146385
0.0023187104516646964
0.00255761041454038
0.02777677876712386
0.048845138581347196
0.0036803742306717067
0.025371017039724986
0.01025628587506104
0.002765966228153332
0.005277411839913076
0.005615262186601555
0.0017685863429519094
0.017667477060567437
0.00022354193946567055
0.003

0.0028179312875636795
0.004182318457632772
0.004935507565411204
0.0035380034202643373
0.028067213389645945
0.005552498619528661
0.0028016191154919133
0.014752944001327113
0.002513917732892021
0.013898434838702733
0.013588525549248054
0.002653794571993049
0.008916328111400227
0.005334232991330445
0.05180789867335226
0.006191731086873394
0.008822405903062833
0.009436835347927056
0.006466245637020511
0.02246346075756449
0.008456556168460642
0.0021477952945257537
0.004834880687992649
0.0027482230836232925
0.009082552757317265
0.01753715346612004
0.0026872604615302766
0.0033128090740747965
0.0032510170836718396
0.029085643663732563
0.01851195362818984
0.007694103408056302
0.0027580950582345646
0.0034348526481695722
0.0005949854589813606
0.005123204366152929
0.0010549701260962779
0.005129903131332446
0.0028086695414740975
0.01122716841901669
0.0030073112295084625
0.0030974844964744342
0.0007368939937255401
0.01058193811539052
0.018479900788398215
0.0023607801272745874
0.010512246312632008
0.

...

0.028167270043688696
0.010159569548236196
0.012406640141664123
0.02013260317481781
0.0454185372572574
0.004380625989373755
0.00024618084653701294
0.008471476457342024
0.016517314545081575
0.004448942434515637
0.005288462715846416
0.021058271087937713
0.0026693838721384444
0.021865419679536487
0.01183515113809471
0.008154682919649514
0.0025106980722810427
0.024186384615411168
0.001177622959303746
0.011395721366572704
0.003447660378490865
0.0076216724412044926
0.027345181963699377
0.013771545994320062
0.01597802700527751
0.006140446346895005
0.020083305249898452
0.011689677770595349
0.0015676429682744254
4.653670109686667e-5
0.000636616459024078
0.005420174429864954
0.00039327149055679283
0.010199525594423184
0.007798715163076058
0.0075380191620654045
0.026116365729503686
0.0010275786176847876
0.005275218168246821
0.01724287608821341
0.005099095617980638
0.014012924496427786
0.007880068753255693
0.01786345684370622
0.009467562624736121
0.009699686862023856
0.0002999114274134368
0.0040547

0.010948387587031176
0.0014720229823240948
0.017025754306820576
0.010490453443915589
0.0010797273563096118
0.0082098918675733
0.0011993365269490693
0.0005536019950681139
0.007701636738897045
0.004766248406341988
0.022038144663330907
0.004917617754722356
0.0004070588459371463
0.010871610111678113
0.015164636336290832
0.0022537068393392377
0.012399263223802257
0.006977767696100663
0.004829508855383427
0.011205570782598017
0.00014820072403512832
0.005111283131770837
0.009201023307511247
0.013115374442086901
0.053620605353194066
0.001170864826907888
0.02646456594993177
0.001429208035109434
0.0048599009153169614
0.00883820113115255
0.014860324272086866
0.024427666129508713
0.049258647050588446
0.02061809694565942
0.004818741612309732
0.0004118752321741237
0.0016342715541559003
0.005092205351006229
0.0048022520443301055
0.003312399879708448
0.0038776613314139867
0.010143556543243263
0.01932756985955336
0.0010891843952511061
0.020083495336686366
0.012078908044210688
0.027194236637339518
0.001

0.02358265244141012
0.0037538713999056743
0.013293006922621897
0.011829716181365437
0.03430517597481527
0.0009886127317637378
0.006193073874867108
0.005353930855375914
0.012804982397031409
0.004810125132842159
0.0070079130573675085
0.008251021233455318
0.0025860902646296807
0.005701792743139703
0.000562432249404651
0.013615443068299227
0.00040879445412445487
0.004596019122879366
0.0063248915023394895
0.009868733119743421
0.018295378572569417
0.001397140990553267
0.03552241276178032
0.0018444708954320232
0.00010442887616127889
0.004226713877956745
0.00024246108682705075
0.00020104486104258431
0.04170969874669979
0.012174581685294653
0.020201160872841142
0.0035261002549754616
0.006114236590207389
0.03450854701588194
0.009147478382166075
0.008409710681384101
0.0006513152454986304
0.003597055610313532
0.002555930409046508
0.009349782407418145
0.0026631501095307395
0.01182301722130812
0.0012567020034343665
0.0006218513361059887
0.0012421353089093854
0.0020677585150144936
0.03815953778141824

0.019031213101808067
0.01693183503535803
0.024064850260702722
0.0014387682871883972
0.0013445257773307146
0.0012100432689604373
0.020543740656280948
0.00551949737012748
0.021948822497549213
0.015222783523167456
0.016346245977733587
0.022433812907009052
0.01126872950199178
0.00046388129523008845
0.007810625546180267
0.0035989996614243652
0.011774964260936103
0.01284968894236535
0.0040026617569187085
0.006797205332478746
0.011879793670786629
0.020069650194867148
0.021565299772798027
0.0012208320306728465
0.030193394797230898
0.0006874845418231637
0.0005403803896919688
0.001159286127194516
0.03426615001696264
0.004393843290979302
0.020493221718780097
0.03579637285089437
0.0009525425663770618
0.004985893041680883
0.016720361127671737
0.04533169852730844
0.020472791068271264
0.004631979623548705
0.004709079832018748
0.002485887144885036
0.008962307938347505
0.0051080305590313615
0.011974560137026144
0.019366993038621452
0.0045624656353004805
0.0004829078585889191
0.024262715564245946
0.0066

0.003888269730341104
0.0065323840254429095
0.06325587549956968
0.10720387232169373
0.010503041642245827
0.001514737014497672
0.010995051417017818
0.016448566778468867
0.0021860510101809
0.009328948930836677
0.005078383950819028
0.00448327742999809
0.012302622596673621
0.027719884053995742
0.0005561879834386003
0.0073188192577195894
0.003507170180776328
0.0010725523380177011
0.002661719563513999
0.005215117917561552
0.010725372227020413
0.008151721931654858
0.005592870969394969
0.0005992136991107472
0.0030096930768723977
9.193308084205037e-5
0.011347538277777318
0.014313022570124252
0.0007564560871216162
0.04031408182063288
0.00543933181659913
0.005463785967835316
0.007468335363540115
0.018485624080381638
0.0024879311289048296
0.040648879568698094
0.0029977206018555282
0.013988176227434325
0.0026449758609410723
0.0023759293917686966
0.005819977263549112
0.0030754414930138745
0.002929054263091753
0.018711775571172415
0.01510067097839891
0.007641111608419463
0.007934070558917077
0.0001049

In [489]:
using MLBase
pred = []
for i in 1:size(D_test[1],1) 
    p = predict!(MT,D_test[1][i,:],10*size(D_test[1],2))
    if p[1] > p[2]
        push!(pred,1)
    else
        push!(pred,2)
    end
end
println(correctrate(D_test[2],convert(Array{Int,1},pred)))
println(unique(pred))

0.675
[2, 1]


In [287]:
a.Intervals

LoadError: [91mtype Mondrian_Node has no field Intervals[39m

In [43]:
D[1][1,2] < a.Intervals[2,1] || D[1][1,2] > a.Intervals[2,2]

false

In [40]:
D[1,2]

LoadError: [91mMethodError: no method matching getindex(::Tuple{Array{Float64,2},Array{Int64,1}}, ::Int64, ::Int64)[39m

In [22]:
Y = [1,2,2,2,1]

5-element Array{Int64,1}:
 1
 2
 2
 2
 1

In [24]:
size(Y[Y.==1],1)

2

In [16]:
ones(2)

2-element Array{Float64,1}:
 1.0
 1.0

In [17]:
exp(2)

7.38905609893065

In [19]:
sum([1,2,3])

6